In [1]:
import sqlite3
import json
import numpy as np
import api

In [2]:
connection = sqlite3.connect('matches.db')
cur = connection.cursor()

In [3]:
heroes = api.call_api_function('IEconDOTA2_570', 'GetHeroes', 'v1')[u'result']
hero_ids = [hero[u'id'] for hero in heroes[u'heroes']]
hero_id_map = {hero_ids[i]: i for i in xrange(len(hero_ids))}
print heroes[u'count']

111


In [4]:
data = []
data_result = []
for match_json in cur.execute('SELECT json FROM match_info'):
    match = json.loads(match_json[0])[u'result']
    if match[u'lobby_type'] not in (0, 2, 5) or match[u'game_mode'] not in (1, 2, 3, 4):
        continue
    if any(player[u'leaver_status'] not in (0, 1) for player in match[u'players']):
        continue
    if any(player[u'hero_id'] == 0 for player  in match[u'players']):
        continue
    if match[u'duration'] <= 60 * 5:
        continue
    # now we've got a good match
    data.append(tuple(hero_id_map[player[u'hero_id']] for player in sorted(match[u'players'], key=lambda x: x[u'player_slot'])))
    data_result.append(match[u'radiant_win'])

In [5]:
len(data) / 111231.

0.40846526597800975

In [6]:
n_heroes = len(hero_id_map)
def process_row(row):
    result = [0] * (2 * n_heroes)
    for i in xrange(5):
        result[row[i]] = 1
    for i in xrange(5, 10):
        result[row[i] + n_heroes] = 1
    return np.array(result)

In [7]:
X = np.array([process_row(row) for row in data])

In [8]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
from sklearn import svm

In [14]:
y = np.array(data_result)

In [15]:
y

array([False,  True,  True, ...,  True, False, False], dtype=bool)

In [19]:
from sklearn import cross_validation

In [20]:
cls = svm.LinearSVC(C=1.0)

In [24]:
np.mean(cross_validation.cross_val_score(cls, X, y, cv=5, scoring='accuracy'))

0.76533007982778845

In [26]:
cls.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [34]:
min(range(222), key=lambda x: cls.coef_.reshape(-1)[x])

89

In [35]:
heroes[u'heroes'][89]

{u'id': 91, u'name': u'npc_dota_hero_wisp'}